# Извлечение именованных сущностей.

Именованные сущности - множество слов (словосочетаний, последовательностей символов), которое представляют инетерес в контексте решаемой практической задачи и которое нужно отделить от остальных слов. Например, имена, фамилии, суммы, статьи кодексов, товары.


Для английского  [SpaCy](https://spacy.io/).

Для русского тэги из pymorphy.

In [1]:
%%capture
!pip install natasha ipymarkup

In [2]:
%%capture
!pip install pymorphy2

In [3]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [7]:
p = morph.parse('Мария')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,femn,Name sing,nomn
True


In [38]:
p = morph.parse('Николаева')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

Тэги -  NOUN,anim,femn,Sgtm,Surn sing,nomn
True


In [10]:
p = morph.parse('Алексеевич')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Patr sing,nomn
True


In [39]:
p = morph.parse('Бибирево')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,neut,Sgtm,Geox sing,nomn
True


In [ ]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Orgn sing,nomn
True


In [40]:
p = morph.parse('МИИТ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Sgtm,Fixd,Abbr,Orgn sing,nomn
True


Рядом стоящие слова одного тэга можно склеить в один.

По-другому: https://github.com/natasha/natasha

In [11]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor
)


from ipymarkup import show_span_box_markup

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [41]:
text = 'Влад Иванов. Андрей Петрович. Алиса Викторовна. Студия Артемия Кондратьева'
doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)
# print(format_json(facts))

In [42]:
matches = names_extractor(text)
spans = [(_.start, _.stop, 'PER') for _ in matches]
show_span_box_markup(text, spans)

In [44]:
text = 'В Москве в музее-заповеднике Коломенское можно найти очень редкие цветы, например Незабудку'

doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)
# print(format_json(facts))

In [46]:
text = 'Московский государственный университет. ВШЭ.'
doc = Doc(text)
doc.segment(segmenter)
doc.tag_ner(ner_tagger)
show_span_box_markup(text, doc.spans)
# print(format_json(facts))

In [47]:
text = 'С 2015 г. по 2016 год. 16 апреля 1997 года. В четверг. 23.04.23'

matches = dates_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)

In [20]:
text = "Он заплатил ему 3 000 000 000 рублей."

matches = money_extractor(text)
spans = [(_.start, _.stop) for _ in matches]
show_span_box_markup(text, spans)
# print(format_json(facts))

In [21]:
text = "Он заплатил ему 3000000,78 рублей."

matches = money_extractor(text)

for m in matches:
  print(m)

Match(start=16, stop=33, fact=Money(amount=3000000.78, currency='RUB'))


Для извлечения специфичных сущностей правила в Yargy.

Из текстов объявлений на авито по категории "Консоли" и извлекаем названия приставок Xbox и Playstation:

In [17]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

<ipython-input-17-fd0b9af24538>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [36]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [24]:
data.shape

(9285, 1)

1) Полные варианты: Xbox 360, Xbox one, Playstation 1,2,3,4.

In [25]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

Сущность Pristavka с двумя атрибутами: название и версия.

In [26]:

Pristavka = fact(
    'Pristavka',
    ['name', 'model']
)


Два списка с вариантами написания Xbox и версий (газзертиры), которые будут сопоставляться с текстом, объединяем в правило:

In [27]:
Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model))


Аналогично для Playstation:

In [28]:
PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model)
    )

Общее правило, которое будет искать плейстешены или иксбоксы и извлекать их как сущность типа Pristavka.

In [29]:
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA) # создаем парсер, которым будем проходить по тексту

In [30]:
matches = []

for sent in data.text[:100]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [31]:
for m in matches[:10]:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2


2) Могут быть подификации вроде Slim, X, S и т.д, но не всегда - правило с .optional() на конце.

In [32]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional())

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA)

In [33]:
matches = []

for sent in data.text[:1000]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [34]:
list(parser.findall('Playstation 3 fdf'))[0].fact

Pristavka(
    name='Playstation',
    model='3',
    version=None
)

In [35]:
for m in matches[:10]:
    print(m.name, m.model)

PS 3
Ps 4
PS 1
PS 3
PlayStation 3
PS 3
Xbox 360
Playstation 3
Ps 4
ps 2


Приведение вариантов названий к нормальному виду (если можно заменить весь газзетир на какое-то одно слово, то можно просто добавить в интерпретации значение .сonst('something')):

In [ ]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model),
    morph_pipeline(['s', 'x', 'e']).interpretation(Pristavka.version).optional())

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    morph_pipeline(['1', '2', '3', '4']).interpretation(Pristavka.model),
    morph_pipeline(['Slim', 'SuperSlim', 'слим']).interpretation(Pristavka.version).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA)

In [ ]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [ ]:
for m in matches[:10]:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2


Когда у каждого и слов в газзетире есть своя нормальная форма, можно сделать газзетир словарём, где ключи это нужные формы, а значения - нормальные формы. А в интерпретации к .normalized() добавить .custom() и через него дергать нужную правильную форму.

In [ ]:
Pristavka = fact(
    'Pristavka',
    ['name', 'model','version']
)

VERSIONS = {
    'super slim': 'SuperSlim',
    'superslim': 'SuperSlim',
    'slim': 'Slim',
    'fat': 'Fat',
    'pro': 'PRO',
    'vita': 'VITA'
}

Xbox = rule(
    morph_pipeline(['Xbox', 'X box', "Иксбокс"]).interpretation(Pristavka.name.const('Xbox')),
    morph_pipeline(['360', 'one']).interpretation(Pristavka.model.normalized()),
    in_caseless('sxe').interpretation(Pristavka.version.normalized()).optional())

PS = rule(
    morph_pipeline(['Playstation', 'Play station', 'PS']).interpretation(Pristavka.name.const('Playstation')),
    # для простоты можно написать вот так
    in_('1234').interpretation(Pristavka.model.normalized()),
    #изменения вот тут                                                      вот тут дергаем правильную форму
    caseless_pipeline(VERSIONS).interpretation(Pristavka.version.normalized().custom(VERSIONS.get)).optional()
    )
PRISTAVKA = or_(PS, Xbox).interpretation(Pristavka)

parser = Parser(PRISTAVKA)

In [ ]:
list(parser.findall('PS 3 SlIm'))[0].fact

Pristavka(
    name='Playstation',
    model='3',
    version='Slim'
)

In [ ]:
list(parser.findall('XbOx 360 X'))[0].fact

Pristavka(
    name='Xbox',
    model='360',
    version='x'
)

In [ ]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [ ]:
for m in matches:
    print(m.name, m.model)

Playstation 3
Playstation 4
Playstation 1
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 3
Playstation 4
Playstation 2
Xbox 360
Playstation 4
Playstation 3
Playstation 4
Playstation 3
Xbox 360
Xbox one
Playstation 4
Xbox 360
Xbox one
Playstation 3
Xbox 360
Playstation 4
Xbox 360
Playstation 3
Xbox 360
Playstation 4
Xbox one
Playstation 4
Playstation 4
Xbox 360
Xbox 360
Playstation 3
Xbox 360
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Playstation 4
Playstation 4
Playstation 4
Xbox 360
Xbox one
Xbox 360
Playstation 3
Xbox 360
Xbox 360
Playstation 3
Playstation 2
Playstation 3
Playstation 3
Playstation 3
Playstation 3
Xbox 360
Playstation 4
Xbox one
Xbox one
Playstation 2
Playstation 4
Xbox one
Playstation 4
Playstation 4
Playstation 3
Playstation 3
Xbox one
Playstation 4
